## Object Oriented Programming - Quick and dirty Introduction

In this session, we go through a toy example of object oriented programming in python and then look at the skeleton class of a document. Clearly, this topic is vast and requires a much more through treatment than being given here. But I find it more useful to introduce you to concepts because we need them rather than because they exist. 

#### Why will we use object oriented programming in this class?
Amongst the many many reasons that OOP makes sense, for us the most important one is:

1. __Reusability__
We want to be able to write all our code for every document once and then call it again and again. Having stand alone functinons not only increases the amount of _house keeping_ but will also require you to change code at different places everytime your requirements change.

2. __Encapsulation__
Remember that you will have hundreds of documents and will have to define properties for each one of them. For example, how long is every document or what are its tokens. OOP allows you to define all of these properties at one place and then the details of these implementations can be hidden.

#### Important Terminology

1. __Class__: Is _dna_ or _blueprint_ for an object that has to be modeled. Contains attributes that contins its properties and methods that characterize its behaviors.

2. __Instance__: An individual object of a class.

3. __Instantiation__: The creation of an instance of a class.

4. __Object__: A unique instance of a class. An object comprises both data members (class variables and instance variables) and methods.

5. __Class variable__: A variable that is shared by all instances of a class. They are defined within a class but outside any of the class's methods

6. __Instance variable__: A variable that is defined inside a method and belongs only to the current instance of a class.

7. __Data member__: A class variable or instance variable that holds data associated with a class and its objects.

8. __Method__ : A function that is defined in a class definition. 

So all the chit chat aside lets get down to it 

In [1]:
import numpy as np
from nltk.tokenize import wordpunct_tokenize
from nltk import PorterStemmer
import re

"""
This is a class sherlock. 
Notice how it is defined with the keyword `class` and a name that begins with a capital letter
"""
class Document():
    
    """ The Doc class rpresents a class of individul documents
    
    """
    
    def __init__(self, speech_year, speech_pres, speech_text):
        """
        The __init__ method is called everytime an object is instantiated.
        This is where you will define all the properties of the object that it must have
        when it is `born`.
        """
        
        #These are data members
        self.year = speech_year
        self.pres = speech_pres
        self.text = speech_text.lower()
        self.tokens = np.array(wordpunct_tokenize(self.text))
        
        
        
    def token_clean(self,length):

        """ 
        description: strip out non-alpha tokens and tokens of length > 'length'
        input: length: cut off length 
        """

        self.tokens = np.array([t for t in self.tokens if (t.isalpha() and len(t) > length)])


    def stopword_remove(self, stopwords):

        """
        description: Remove stopwords from tokens.
        input: stopwords: a suitable list of stopwords
        """
 
        self.tokens = np.array([t for t in self.tokens if t not in stopwords])


    def stem(self):

        """
        description: Stem tokens with Porter Stemmer.
        """
        
        self.tokens = np.array([PorterStemmer().stem(t) for t in self.tokens])
        
    def demo_self():
        print 'this will error out'


### Self
Notice the `self` keyword that is present all over the place in the class. `self` tells python which object it needs to work with. 

1. Therefore every method should have a `self` parameter specified.
2. Notice that when the method is called the reference to the object is passed implicitly.
3. Every data member of the class needs to be referred to using `self`. 

In [2]:
#Instantiating an object.  
speech1 = Document('1986', 'Rooster', 'this is a chicken')
print speech1

#Accessing data members
print speech1.tokens

try:
    speech1.demo_self()
except Exception as ex:
    print ex

<__main__.Document instance at 0x7f354d389128>
['this' 'is' 'a' 'chicken']
demo_self() takes no arguments (1 given)


### Demo of usefulness of classes

Use this [link](http://www.codeskulptor.org/#user41_X9owzYCGupNiKYr.py) to see how useful classes can be. The link leads to a code I wrote two years ago that implements a very naive version of the game Asteroids. 

## A skeleton class structure for documents

In [3]:
import numpy as np
import codecs
import nltk
import re
import math 
from nltk.tokenize import wordpunct_tokenize
from nltk import PorterStemmer
from itertools import repeat


class Corpus():
    
    """ 
    The Corpus class represents a document collection
     
    """
    def __init__(self, doc_data, stopword_file, clean_length):
        """
        Notice that the __init__ method is invoked everytime an object of the class
        is instantiated
        """
        

        #Initialise documents by invoking the appropriate class
        self.docs = [Document(doc[0], doc[1], doc[2]) for doc in doc_data] 
        
        self.N = len(self.docs)
        self.clean_length = clean_length
        
        #get a list of stopwords
        self.create_stopwords(stopword_file, clean_length)
        
        #stopword removal, token cleaning and stemming to docs
        self.clean_docs(2)
        
        #create vocabulary
        self.corpus_tokens()
        
    def clean_docs(self, length):
        """ 
        Applies stopword removal, token cleaning and stemming to docs
        """
        for doc in self.docs:
            doc.token_clean(length)
            doc.stopword_remove(self.stopwords)
            doc.stem()        
    
    def create_stopwords(self, stopword_file, length):
        """
        description: parses a file of stowords, removes words of length 'length' and 
        stems it
        input: length: cutoff length for words
               stopword_file: stopwords file to parse
        """
        
        with codecs.open(stopword_file,'r','utf-8') as f: raw = f.read()
        
        self.stopwords = (np.array([PorterStemmer().stem(word) 
                                    for word in list(raw.splitlines()) if len(word) > length]))
        
     
    def corpus_tokens(self):
        """
        description: create a set of all all tokens or in other words a vocabulary
        """
        
        #initialise an empty set
        self.token_set = set()
        for doc in self.docs:
            self.token_set = self.token_set.union(doc.tokens) 
            
    def document_term_matrix(self):
        """
        description:  returns a D by V array of frequency counts
        """  
        # subroutine: computes the counts of each vocabulary in the document
        def counts(doc):
            # initialize a matrix
            term_mat = [0]*len(self.token_set)
            for token in doc.tokens:
                term_mat[list(self.token_set).index(token)] = term_mat[list(self.token_set).index(token)] + 1
            return term_mat;
            
        self.doc_term_matrix = []
        
        for doc in self.docs:
            self.doc_term_matrix.append(counts(doc))


      
    def tf_idf(self):
        """
        description:  returns a D by V array of tf-idf scores
        """
        # Compute inverse document frequency 
        idf = [0]*len(self.token_set)
        for token in self.token_set:
            ind = 0
            for doc in self.docs:
                if token in doc.tokens:
                    ind += 1 
            idf[list(self.token_set).index(token)] = math.log(self.N/ind)
        
        # Create a subroutine that computes tf_idf for one document
        def tfidf(doc):
            term_mat = [0]*len(self.token_set)
            for token in doc.tokens:
                term_mat[list(self.token_set).index(token)] = term_mat[list(self.token_set).index(token)] + 1 
        
            for i,term in enumerate(term_mat):
                if term != 0:
                    term_mat[i] = (1 + math.log(term)) * idf[i]
            return term_mat;
        
        #tf_idf
        self.tf_idf_matrix = []
        for doc in self.docs:
            self.tf_idf_matrix.append(tfidf(doc))
            
            
        
    def dict_rank(self, n, dictionary, token_repr):
        """
        description:  returns the top n documents based on a given dictionary and represenation of tokens
        """
        if token_repr == "tf-idf":
            self.tf_idf()
            representation = self.tf_idf_matrix
            
        if token_repr == "doc-term":
            self.document_term_matrix()
            representation = self.doc_term_matrix
            
        # Return top n docs based on dictionary given
        
        score = []
        x=self.token_set
        x=list(x)
        for token in x: 
            try:
                score.append(dictionary[token])
            except: 
                score.append(0)

        # get a vector with all the scores in order
        score=[int(x) for x in score]
        rank = {}
        elements=range(len(representation))
        for i in elements:
            rank["DOC-"+str(i)] = np.dot(score,representation[i])
        
            
            
        # Get sorted view of the keys.
        s = sorted(rank, key=rank.get, reverse=True)[0:(n-1)]
        
        ranking = {}
        for key in s:
            ranking[key] =  rank[key]
        
        return ranking 



In [4]:
class Document():
    
    """ The Doc class rpresents a class of individul documents
    
    """
    
    def __init__(self, speech_year, speech_pres, speech_text):
        self.year = speech_year
        self.pres = speech_pres
        self.text = speech_text.lower()
        self.tokens = np.array(wordpunct_tokenize(self.text))
        
        
        
    def token_clean(self,length):

        """ 
        description: strip out non-alpha tokens and tokens of length > 'length'
        input: length: cut off length 
        """

        self.tokens = np.array([t for t in self.tokens if (t.isalpha() and len(t) > length)])


    def stopword_remove(self, stopwords):

        """
        description: Remove stopwords from tokens.
        input: stopwords: a suitable list of stopwords
        """

        
        self.tokens = np.array([t for t in self.tokens if t not in stopwords])


    def stem(self):

        """
        description: Stem tokens with Porter Stemmer.
        """
        
        self.tokens = np.array([PorterStemmer().stem(t) for t in self.tokens])


We will use the presedential speech dataset to demonstrate how the class works 

In [5]:
def parse_text(textraw, regex):
    """takes raw string and performs two operations
    1. Breaks text into a list of speech, president and speech
    2. breaks speech into paragraphs
    """
    prs_yr_spch_reg = re.compile(regex, re.MULTILINE|re.DOTALL)
    
    #Each tuple contains the year, last ane of the president and the speech text
    prs_yr_spch = prs_yr_spch_reg.findall(textraw)
    
    #convert immutabe tuple to mutable list
    prs_yr_spch = [list(tup) for tup in prs_yr_spch]
    
    for i in range(len(prs_yr_spch)):
        prs_yr_spch[i][2] = prs_yr_spch[i][2].replace('\n', '')
    
    #sort
    prs_yr_spch.sort()
    
    return(prs_yr_spch)

In [10]:
#text = open("/Users/ainalopez/Downloads/text_mining-master/data/pres_speech/sou_all.txt", 'r').read()
text = open("/home/yaroslav/Projects/text_mining/data/pres_speech/sou_all.txt", 'r').read()
regex = "_(\d{4}).*?_[a-zA-Z]+.*?_[a-zA-Z]+.*?_([a-zA-Z]+)_\*+(\\n{2}.*?)\\n{3}"
pres_speech_list = parse_text(text, regex)
# Select only some documents
pres_speech_list = pres_speech_list[230:235]


In [11]:
#Instantite the corpus class
corpus = Corpus(pres_speech_list, '/home/yaroslav/Projects/text_mining/data/stopwords/stopwords.txt', 2)


In [12]:
# Import dictionary from excel file
import pandas as pd
df = pd.read_excel("/home/yaroslav/Dropbox/BGSE/3rd Term/Text Mining/LoughranMcDonald_MasterDictionary_2014.xlsx", skiprows=0)
w = df['Word']
words = [str(x).lower() for x in df['Word']]
words=[PorterStemmer().stem(t) for t in words]
score = [str(x).lower() for x in df['Positive']] # or any other method
dictionary=dict(zip(words,score))


In [13]:

# Applying dict_rank function
X1 = corpus.dict_rank(10, dictionary,"doc-term")
X2 = corpus.dict_rank(10, dictionary,"tf-idf")

In [14]:
print X1
print X2

{'DOC-4': 136612, 'DOC-0': 130585, 'DOC-1': 136612, 'DOC-2': 122549, 'DOC-3': 94423}
{'DOC-4': 24026.058048305684, 'DOC-0': 16554.292251414543, 'DOC-1': 13885.554901304156, 'DOC-2': 19325.300257919924, 'DOC-3': 9251.7869036500761}


In [31]:
# Exercise 2:

from pandas import DataFrame

# Get list of presidents
presidents = [presi.pres for presi in corpus.docs]
pr1 = list(X1.keys())
pr2 = list(X2.keys())

pr1=[x[4:] for x in pr1]
pr2=[x[4:] for x in pr2]

pr1=[int(x) for x in pr1]
pr2=[int(x) for x in pr2]

pres1 = [presidents[i] for i in pr1]
pres2 = [presidents[i] for i in pr2]

scores1 = list(X1.values())
scores2 = list(X2.values())
print scores1

df1 = DataFrame({
        'president': pres1,
        'score1': scores1
        })

df2 = DataFrame({
        'president': pres2,
        'score1': scores2
        })


g1=df1.groupby(['president']).mean()
g2=df2.groupby(['president']).mean()
#g = df1.groupby('president').mean().sort_values(ascending=False)
#most_rated = lens.groupby('title').size().sort_values(ascending=False)[:25]
print g1
print g2


[136612, 130585, 136612, 122549, 94423]
           score1
president        
Obama      124156
                 score1
president              
Obama      16608.598473


In [23]:
pr1 = list(X1.keys())
pr2 = list(X2.keys())

pr1=[x[4:] for x in pr1]
pr2=[x[4:] for x in pr2]

pr1=[int(x) for x in pr1]
pr2=[int(x) for x in pr2]

presidents = [presi.pres for presi in corpus.docs]
pres1 = [presidents[i] for i in pr1]
pres2 = [presidents[i] for i in pr2]

print pres1

[4, 0, 1, 2, 3]
['Obama', 'Obama', 'Obama', 'Obama', 'Obama']
